In [ ]:
pip install researchpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\src")
    sys.path.append(module_path+"\\References")

import data_manipulation as dm 
import data_visualization as dv 
import data_dictionary as dd
import data_preprocessing as dp

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Dropping low variance Features

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [ ]:
file_name = 'imputed_data'
data = pd.read_excel('/content/'+file_name+'.xlsx')
data.head()

,Organization type,Role in organization,Size of organization,Size of IT department,Customer organization type,Actual duration,Development type,Application domain,Object points,Other sizing method,Actual effort,Contract maturity,Government policy impact,Economic instability impact,Organization management structure clarity,Developer hiring policy,Developer incentives policy,Developer training,Development team management,Top management support,Top management opinion of previous system,Clarity of manual system,User resistance,User computer experience,Users stability,Requirment stability,Requirements flexibility,Project manager experience,Consultant availability,DBMS expert availability,Precedentedness,Software tool experience,Programmers experience in programming language,Programmers capability,Analysts capability,Team selection,Team size,Dedicated team members,Daily working hours,Team contracts,Team continuity,Team cohesion,Income satisfaction,Schedule quality,Development environment adequacy,Tool availability,Methodology,Multiple programing languages,Programming language used,DBMS used,Technical stability,Open source software,Level of outsourcing,Degree of software reuse,Degree of risk management,Use of standards,Process reengineering,Requirement accuracy level,Technical documentation,Comments within the code,User manual,Required reusability,Performance requirements,Product complexity,Security requirements,Reliability requirements,Specified H/W
0,1,1,16,7,13,3.0,1,7,100,2,3168.0,8,4,1,2,1,8,3,4,4,1,2,1.75,2,1.888889,2.587302,4.555556,2,3,0,2,2,2,2,1,3,6,6,8,1,16.666667,2,3,2,1,7,2,1,1,1,1,1,3,5,3,3,1.888889,1,1,3,1,1,1,2,2,3,1
1,5,1,2,1,2,3.0,1,3,19,2,1584.0,4,4,3,1,2,6,3,7,4,1,4,2.50,2,1.888889,2.904762,4.555556,1,3,0,2,5,2,2,2,3,3,3,8,1,0.000000,6,4,2,1,8,1,2,5,1,1,1,3,5,3,3,4.555556,2,2,2,1,4,1,1,4,2,1
2,5,3,2,2,3,5.0,1,3,64,3,5280.0,4,5,2,3,4,6,3,6,1,0,6,2.00,2,1.888889,1.000000,1.888889,2,2,1,4,1,1,2,2,2,6,6,8,1,0.000000,5,2,1,1,8,3,1,3,1,1,0,3,5,2,3,1.444444,3,2,2,2,3,4,3,5,2,3
3,4,2,3,2,4,6.0,1,7,246,5,5280.0,2,4,1,1,1,4,3,6,4,1,3,3.25,2,5.000000,2.507937,1.000000,1,1,1,4,1,1,4,4,2,6,4,8,2,0.000000,1,2,1,1,4,2,1,4,2,2,0,1,3,1,2,1.444444,1,1,3,4,3,2,4,3,4,2
4,4,2,3,2,4,24.0,1,7,420,5,38016.0,3,4,3,1,2,3,3,2,4,1,3,2.75,2,1.888889,4.650794,1.000000,1,2,1,4,2,1,4,4,3,9,9,8,1,44.444444,2,3,1,1,4,2,3,24,2,1,0,3,3,1,3,1.000000,1,1,3,4,3,4,3,3,4,2


In [ ]:
data.shape

(120, 67)

In [ ]:
variances = np.round(data.var().sort_values(ascending = False), 2)
variances[20:]

Users stability                                   1.83
DBMS used                                         1.83
User manual                                       1.81
Software tool experience                          1.71
Methodology                                       1.63
Developer hiring policy                           1.60
Economic instability impact                       1.57
Degree of software reuse                          1.53
Requirement accuracy level                        1.47
Required reusability                              1.44
Process reengineering                             1.38
Reliability requirements                          1.34
Programmers experience in programming language    1.34
Top management support                            1.28
Government policy impact                          1.23
Technical documentation                           1.23
Requirment stability                              1.22
Precedentedness                                   1.12
Role in or

In [ ]:
data.drop(list(variances[variances < 0.3].index), axis = 1, inplace = True)

# F regression test for numerical columns

In [ ]:
from sklearn.feature_selection import f_regression

In [ ]:
categorical_columns = [col for col in dd.categorical_cols_names() if col in data.columns]
numerical_columns = [col for col in data.columns if col not in categorical_columns]
numerical_columns.remove('Actual effort')

In [ ]:
scores = {'Score': f_regression(data[numerical_columns], data['Actual effort'])[0], 'p-value': np.round(f_regression(data[numerical_columns], data['Actual effort'])[1], 4)}
scores = pd.DataFrame(scores, index = numerical_columns).sort_values('p-value', ascending = True)
scores.head(20)

,Score,p-value
Actual duration,21.600395,0.0000
Object points,103.135779,0.0000
Dedicated team members,280.423008,0.0000
Team size,287.441018,0.0000
Requirment stability,11.883554,0.0008
Technical stability,9.598140,0.0024
Team cohesion,9.437774,0.0026
Tool availability,8.008831,0.0055
Economic instability impact,7.151680,0.0086
Developer incentives policy,7.073320,0.0089


In [ ]:
columns_to_drop = scores[scores['p-value'] > 0.05].index
data.drop(list(columns_to_drop), axis = 1, inplace = True)
data.shape

(120, 46)

# Anova test for categorical columns

In [ ]:
from sklearn.feature_selection import f_classif

In [ ]:
scores = {'Score': f_classif(data[categorical_columns], data['Actual effort'])[0], 'p-value': np.round(f_classif(data[categorical_columns], data['Actual effort'])[1], 4)}
scores = pd.DataFrame(scores, index = categorical_columns).sort_values('p-value', ascending = True)
scores.head(10)

,Score,p-value
Role in organization,3.962587,0.0000
Development type,2.947634,0.0001
Size of organization,1.908943,0.0087
Requirement accuracy level,1.791512,0.0158
Comments within the code,1.502720,0.0657
DBMS used,1.486243,0.0710
Programmers experience in programming language,1.486493,0.0710
Required reusability,1.467147,0.0777
Size of IT department,1.392763,0.1097
Programming language used,1.351186,0.1322


In [ ]:
columns_to_drop = scores[scores['p-value'] > 0.05].index
data.drop(list(columns_to_drop), axis = 1, inplace = True)

In [ ]:
data.head()

,Role in organization,Size of organization,Actual duration,Development type,Object points,Actual effort,Contract maturity,Economic instability impact,Developer incentives policy,User resistance,Users stability,Requirment stability,Precedentedness,Team size,Dedicated team members,Team cohesion,Development environment adequacy,Tool availability,Technical stability,Requirement accuracy level
0,1,16,3.0,1,100,3168.0,8,1,8,1.75,1.888889,2.587302,2,6,6,2,1,7,1,1
1,1,2,3.0,1,19,1584.0,4,3,6,2.50,1.888889,2.904762,2,3,3,6,1,8,1,2
2,3,2,5.0,1,64,5280.0,4,2,6,2.00,1.888889,1.000000,4,6,6,5,1,8,1,3
3,2,3,6.0,1,246,5280.0,2,1,4,3.25,5.000000,2.507937,4,6,4,1,1,4,2,1
4,2,3,24.0,1,420,38016.0,3,3,3,2.75,1.888889,4.650794,4,9,9,2,1,4,1,1


# OneHot Encoding

In [ ]:
categorical_cols = [col for col in data.columns if col in dd.categorical_cols_names()]
for col in categorical_cols:
  data[col].replace(dd.values_meaning(col), inplace = True)

In [ ]:
onehot_encoder = OneHotEncoder()
transformed = onehot_encoder.fit_transform(data[['Role in organization', 'Development type', 'Size of organization', 'Requirement accuracy level']])
one_hot_columns = [col[3::] for col in onehot_encoder.get_feature_names()]
#Create a Pandas DataFrame of the hot encoded column
ohe_df = pd.DataFrame(data = transformed.todense(), columns = one_hot_columns)
df = pd.concat([data.drop(['Role in organization', 'Development type', 'Size of organization', 'Requirement accuracy level'], axis = 1), ohe_df], axis = 1)
df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Actual duration,Object points,Actual effort,Contract maturity,Economic instability impact,Developer incentives policy,User resistance,Users stability,Requirment stability,Precedentedness,Team size,Dedicated team members,Team cohesion,Development environment adequacy,Tool availability,Technical stability,Company manager,Developer,Planning coordinator,Project manager,System administrator,Technical consultant,Technical manager,Customization of imported software,Modifying existing software,New software development,Upgrading existing software,1-5,101-150,11-20,151-200,21-30,31-40,351-400,41-50,51-100,6-10,>500,Accurate requirements specifications used to develop the software system,Inaccurate requirements specifications and required re-programming the software system,Inaccurate requirements specifications and required the re-analysis of the software requirements,Inaccurate requirements specifications and required the re-design of the software system
0,3.0,100,3168.0,8,1,8,1.75,1.888889,2.587302,2,6,6,2,1,7,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,3.0,19,1584.0,4,3,6,2.50,1.888889,2.904762,2,3,3,6,1,8,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,5.0,64,5280.0,4,2,6,2.00,1.888889,1.000000,4,6,6,5,1,8,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,6.0,246,5280.0,2,1,4,3.25,5.000000,2.507937,4,6,4,1,1,4,2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,24.0,420,38016.0,3,3,3,2.75,1.888889,4.650794,4,9,9,2,1,4,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# Polynomial Transformation Process

In [ ]:
target = df['Actual effort']
df.drop(['Actual effort'], axis = 1, inplace = True)
original_columns = list(df.columns)
df.shape

(120, 41)

In [ ]:
df = dp.PolynomialFeatures_labeled(df, 3)

In [ ]:
for col in original_columns:
  df.drop([col+'**2', col+'**3'], axis = 1, inplace = True)
df.shape

(120, 13162)

In [ ]:
df.drop([col for col in df.columns if df[col].min() == 0 and df[col].max() == 0], axis = 1, inplace = True)

In [ ]:
corrs = df.corr() == 1 
boolean = corrs.sum().sort_values(ascending = False) > 1 
indexes = boolean[boolean == True].index
for col in indexes:
  if col not in df.columns:
    pass
  else:
    corrs = df.corr()[col].sort_values(ascending = False) 
    indexes2 = list(corrs[corrs == 1].index)
    if len(indexes2) > 1:
      indexes2.remove(col)
      df.drop(indexes2, axis = 1, inplace = True)

In [ ]:
df = pd.concat([df, target], axis = 1)

In [ ]:
df.head()

,Constant Term,Actual duration**1,Object points**1,Contract maturity**1,Economic instability impact**1,Developer incentives policy**1,User resistance**1,Users stability**1,Requirment stability**1,Precedentedness**1,Team size**1,Dedicated team members**1,Team cohesion**1,Development environment adequacy**1,Tool availability**1,Technical stability**1,Developer**1,Project manager**1,Technical consultant**1,Customization of imported software**1,Modifying existing software**1,New software development**1,Upgrading existing software**1,11-20**1,41-50**1,>500**1,Accurate requirements specifications used to develop the software system**1,Inaccurate requirements specifications and required re-programming the software system**1,Inaccurate requirements specifications and required the re-analysis of the software requirements**1,Actual duration**1 x Object points**1,Actual duration**1 x Contract maturity**1,Actual duration**1 x Economic instability impact**1,Actual duration**1 x Developer incentives policy**1,Actual duration**1 x User resistance**1,Actual duration**1 x Users stability**1,...,Project manager**1 x 21-30**1 x Accurate requirements specifications used to develop the software system**1,Project manager**1 x 51-100**1 x Accurate requirements specifications used to develop the software system**1,Project manager**1 x 6-10**1 x Accurate requirements specifications used to develop the software system**1,Project manager**1 x 6-10**1 x Inaccurate requirements specifications and required re-programming the software system**1,Project manager**1 x 6-10**1 x Inaccurate requirements specifications and required the re-analysis of the software requirements**1,Project manager**1 x 6-10**1 x Inaccurate requirements specifications and required the re-design of the software system**1,Project manager**1 x >500**1 x Accurate requirements specifications used to develop the software system**1,Project manager**1 x >500**1 x Inaccurate requirements specifications and required re-programming the software system**1,Project manager**1 x >500**1 x Inaccurate requirements specifications and required the re-analysis of the software requirements**1,Project manager**1 x Inaccurate requirements specifications and required the re-design of the software system**2,Technical consultant**1 x New software development**1 x >500**1,Technical consultant**1 x New software development**1 x Accurate requirements specifications used to develop the software system**1,Technical consultant**1 x >500**1 x Accurate requirements specifications used to develop the software system**1,Customization of imported software**1 x 11-20**1 x Accurate requirements specifications used to develop the software system**1,Customization of imported software**1 x >500**1 x Accurate requirements specifications used to develop the software system**1,Customization of imported software**1 x >500**1 x Inaccurate requirements specifications and required re-programming the software system**1,Modifying existing software**1 x 1-5**1 x Accurate requirements specifications used to develop the software system**1,New software development**2 x 21-30**1,New software development**2 x 6-10**1,New software development**2 x Inaccurate requirements specifications and required the re-design of the software system**1,New software development**1 x 1-5**2,New software development**1 x 1-5**1 x Accurate requirements specifications used to develop the software system**1,New software development**1 x 11-20**1 x Accurate requirements specifications used to develop the software system**1,New software development**1 x 11-20**1 x Inaccurate requirements specifications and required re-programming the software system**1,New software development**1 x 21-30**1 x Accurate requirements specifications used to develop the software system**1,New software development**1 x 51-100**2,New software development**1 x 6-10**1 x Inaccurate requirements specifications and required re-programming the software system**1,New software development**1 x

# Saving data

In [ ]:
df.to_excel('/content/preprocessed_data.xlsx')